In [1]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

/aiffel/aiffel/aimmo/mmdetection


In [2]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [3]:
# # 완디비 로그인
wandb.login(key= '')

wandb: Currently logged in as: choihoonsung33 (tesk-b). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [4]:
wandb.init(project="test-project", entity="tesk-b")

In [5]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_txt')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [6]:
# # We download the pre-trained checkpoints for inference and finetuning.
# !mkdir checkpoints
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_l_8x8_300e_coco/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth \
#       -O checkpoints/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth

In [7]:
config = 'configs/yolox/yolox_l_8x8_300e_coco.py'
cfg = Config.fromfile(config)


cfg.runner['max_epochs'] = 12

cfg.log_config['interval'] = 1
cfg.model.bbox_head['num_classes'] = 4

cfg.checkpoint_config.interval =1 
cfg.log_config.interval = 10
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'test-project', 'entity' : 'tesk-b', 'name' : 'yolo-x'})] 

cfg.load_from = 'checkpoints/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth'
#cfg.resume_from=


cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.train_dataset.dataset.type = 'Aimmo'
cfg.train_dataset.dataset.ann_file = 'data/middle_train.txt'
cfg.train_dataset.dataset.img_prefix = 'data/image'



cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 0

cfg.data.train.dataset.type = 'Aimmo'
cfg.data.train.dataset.ann_file = 'data/middle_train.txt'
cfg.data.train.dataset.img_prefix = 'data/image'

cfg.data.val.type = 'Aimmo'
cfg.data.val.ann_file = 'data/middle_val.txt'
cfg.data.val.img_prefix = 'data/image'

cfg.data.test.type = 'Aimmo'
cfg.data.test.ann_file = 'data/middle_test.txt'
cfg.data.test.img_prefix = 'data/test/image'

cfg.max_epochs = 12
cfg.num_last_epochs = 12
cfg.interval = 1

cfg.evaluation.save_best = 'mAP'
cfg.evaluation.interval = 1
cfg.evaluation.metric = 'mAP'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.persistent_workers = False
cfg.device = 'cuda'

cfg.work_dir = 'yolox'
print(cfg.pretty_text)

optimizer = dict(
    type='SGD',
    lr=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup='exp',
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=12)
checkpoint_config = dict(interval=1)
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='test-project', entity='tesk-b', name='yolo-x'))
    ])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,

In [8]:
build_dataset(cfg.data.train)

In [9]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-07 13:44:56,305 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-07 13:44:56,485 - mmdet - INFO - load checkpoint from local path: checkpoints/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth
2022-12-07 13:44:56,749 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.multi_level_conv_cls.0.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([4, 256, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.0.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([4]).
size mismatch for bbox_head.multi_level_conv_cls.1.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([4, 256, 1, 1]).
size mismatch for bbox_head.multi_level_conv_cls.1.bias: copying a param with shape torch.Size([80]) from checkpoint, the 

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2022-12-07 13:45:33,821 - mmdet - INFO - Epoch [1][10/5226]	lr: 1.465e-09, eta: 2 days, 6:16:45, time: 3.116, data_time: 2.404, memory: 6551, loss_cls: 3.6080, loss_bbox: 1.8451, loss_obj: 4.1039, loss: 9.5570
2022-12-07 13:46:03,498 - mmdet - INFO - Epoch [1][20/5226]	lr: 5.858e-09, eta: 2 days, 4:58:33, time: 2.968, data_time: 2.197, memory: 7156, loss_cls: 3.4494, loss_bbox: 1.8871, loss_obj: 3.5756, loss: 8.9122
2022-12-07 13:46:32,482 - mmdet - INFO - Epoch [1][30/5226]	lr: 1.318e-08, eta: 2 days, 4:08:00, time: 2.898, data_time: 2.124, memory: 7156, loss_cls: 3.4786, loss_bbox: 1.821

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.4 task/s, elapsed: 427s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-07 18:07:20,649 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 94793 | 0.906  | 0.837 |
| truck      | 3084  | 25779 | 0.799  | 0.636 |
| bus        | 1786  | 13721 | 0.807  | 0.705 |
| pedestrian | 3063  | 21872 | 0.694  | 0.521 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.675 |
+------------+-------+-------+--------+-------+
2022-12-07 18:07:22,610 - mmdet - INFO - Now best checkpoint is saved as best_mAP_epoch_1.pth.
2022-12-07 18:07:22,612 - mmdet - INFO - Best mAP is 0.6746 at 1 epoch.
2022-12-07 18:07:22,613 - mmdet - INFO - Epoch(val) [1][2323]	AP50: 0.6750, mAP: 0.6746
2022-12-07 18:07:53,538 - mmdet - INFO - Epoch [2][10/5226]	lr: 4.015e-04, eta: 1 day, 22:42:38, time: 3.088, data_time: 2.461, memory: 9758, loss_cls: 0.6638, loss_bbox: 1.8462, loss_obj: 1.1148, loss: 3.6248
2022-12-07 18

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.4 task/s, elapsed: 427s, ETA:     0s
---------------iou_thr: 0.5---------------


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.2 task/s, elapsed: 447s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-09 13:58:44,954 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 72690 | 0.944  | 0.898 |
| truck      | 3084  | 22513 | 0.870  | 0.759 |
| bus        | 1786  | 11524 | 0.896  | 0.823 |
| pedestrian | 3063  | 20046 | 0.741  | 0.575 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.764 |
+------------+-------+-------+--------+-------+
2022-12-09 13:58:45,048 - mmdet - INFO - The previous best checkpoint /aiffel/aiffel/aimmo/mmdetection/yolox/best_mAP_epoch_10.pth was removed
2022-12-09 13:58:46,907 - mmdet - INFO - Now best checkpoint is saved as best_mAP_epoch_11.pth.
2022-12-09 13:58:46,909 - mmdet - INFO - Best mAP is 0.7639 at 11 epoch.
2022-12-09 13:58:46,911 - mmdet - INFO - Epoch(val) [11][2323]	AP50: 0.7640, mAP: 0.7639
2022-12-09 13:59:19,137 - mmdet - INFO - Epoch [12][10/5226]	lr: 5.000e-04, e

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.2 task/s, elapsed: 449s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-09 18:27:19,150 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 74276 | 0.946  | 0.899 |
| truck      | 3084  | 22947 | 0.872  | 0.764 |
| bus        | 1786  | 11634 | 0.896  | 0.826 |
| pedestrian | 3063  | 20480 | 0.749  | 0.583 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.768 |
+------------+-------+-------+--------+-------+
2022-12-09 18:27:19,257 - mmdet - INFO - The previous best checkpoint /aiffel/aiffel/aimmo/mmdetection/yolox/best_mAP_epoch_11.pth was removed
2022-12-09 18:27:21,124 - mmdet - INFO - Now best checkpoint is saved as best_mAP_epoch_12.pth.
2022-12-09 18:27:21,126 - mmdet - INFO - Best mAP is 0.7679 at 12 epoch.
2022-12-09 18:27:21,127 - mmdet - INFO - Epoch(val) [12][2323]	AP50: 0.7680, mAP: 0.7679


learning_rate,▁▁▁▁▁▂▂▂▃▃▄▄▅▆▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▆█▅▅█▆▆▆▆▇█▇█▇▇▄▆▄▃▄▂▃▃▃▂▂▃▂▃▃▁▂▁▂▄▁▁▂
train/loss_bbox,▄▄▅█▃▅█▆▆▅▆▇█▇█▆▆▃▆▃▃▅▁▄▄▄▂▂▃▂▃▃▂▂▁▃▃▂▁▂
train/loss_cls,█▅▄▅▄▄▄▃▄▄▃▄▅▄▅▄▃▂▃▃▃▃▁▂▂▂▃▁▂▂▂▂▁▂▁▂▂▁▂▂
train/loss_obj,█▅▅▆▆▄█▅▅▆▅▆▆▆▆▇▆▄▅▄▃▄▃▂▂▃▂▂▃▁▂▂▁▂▂▁▄▁▁▂
val/AP50,▂▁▅▆▆▆▆▇▇▇██
val/mAP,▂▁▅▆▆▆▆▇▇▇██
learning_rate,0.0005
momentum,0.9
train/loss,3.08244
